In [2]:
with open("mShakespere.txt" ,'r' , encoding='utf-8') as f:
    text = f.read()

In [3]:
print(len(text))

1115394


In [4]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [6]:
stoi = { ch:i for i,ch in enumerate(chars)}
itos = { i:ch for i,ch in enumerate(chars)}

def encoder(string):
    l = []
    for ch in string:
        l.append(stoi[ch])
    return l

def decoder(lst):
    s =''
    for val in lst:
        s += itos[val]   
    return s

decoder(encoder('hello'))

'hello'

In [7]:
import torch
data = torch.tensor(encoder(text),dtype=torch.long)
print(data.shape)
data[:100]

torch.Size([1115394])


tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])

In [8]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]
print('len of train',len(train_data) ,'length of val',len(val_data))

len of train 1003854 length of val 111540


In [9]:
block_size = 83
train_data[ :block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47])

In [10]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]

    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47, 58]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47]) the target: 64
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64]) the target: 43
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43]) the target: 52
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52]) the target: 10
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10]) the target: 0
when input is tensor([18, 47, 56, 57, 58,  1, 15, 

In [11]:
torch.randint(len(data) - block_size ,(32,)).shape #


torch.Size([32])

In [12]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53

In [13]:
import torch 
import torch.nn as nn
from torch.nn import functional as F 



class BigramLanguageModel(nn.Module):

    def __init__(self , vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)

    
    def forward(self , idx , targets=None):

        logits = self.token_embedding_table(idx)

        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits,targets)

        return logits , loss
    

    def generate(self , idx , max_new_tokens):
        for _ in range(max_new_tokens):
            logits , loss = self(idx)
            logits = logits[: , -1 ,:]

            probs = F.softmax(logits ,dim=1)

            idx_next = torch.multinomial(probs , num_samples=1)

            idx = torch.cat((idx , idx_next),dim=1)
        return idx
            

        
    
m = BigramLanguageModel(vocab_size)
logits ,loss = m(xb,yb)
print(logits.shape)
print(loss)
    
decoder(m.generate(idx = torch.zeros((1 , 1), dtype = torch.long) , max_new_tokens=100)[0].tolist())

torch.Size([32, 65])
tensor(5.0364, grad_fn=<NllLossBackward0>)


"\nl-QYjt'CL?jLDuQcLzy'RIo;'KdhpV\nvLixa,nswYZwLEPS'ptIZqOZJ$CA$zy-QTkeMk x.gQSFCLg!iW3fO!3DGXAqTsq3pdgq"

In [14]:
optimizer = torch.optim.AdamW(m.parameters() , lr = 1e-3)


In [15]:
len(text)

1115394

In [16]:
# Read the text file and count the spaces
def count_spaces(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()
            space_count = text.count(' ')
        return space_count
    except FileNotFoundError:
        return "File not found."

# Replace 'your_text_file.txt' with the path to your text file
file_path = 'mShakespere.txt'

spaces = count_spaces(file_path)
print(f"The number of spaces in the text file is: {spaces}")


The number of spaces in the text file is: 169892


In [17]:
batch_size = 32

for steps in range(10000):

    xb ,yb = get_batch('train')

    logits , loss = m(xb ,yb)
    
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    if steps % 1000 ==0:
        print(loss.item())



4.658271312713623
3.7861011028289795
3.1342856884002686
2.658357620239258
2.6050984859466553
2.5792124271392822
2.5388400554656982
2.433148145675659
2.4081239700317383
2.496692657470703


In [18]:
print(decoder(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


Ong h hasbe pave pirance
RDe hicomyonthar's
PES:
AKEd ith henourzincenonthioneir thondy, y heltieiengerofo'dsssit ey
KINld pe wither vouprroutherccnohathe; d!
My hind tt hinig t ouchos tes; st yo hind wotte grotonear 'so itJas
Waketancotha:
h hay.JUCLUKn prids, r loncave w hollular s O:
HIs; ht anjx?

DUThinqunt.

LaZEESTEORDY:
h l.
KEONGBUCHandspo be y,-JZNEEYowddy scace, tridesar, wnl'shenous s ls, theresseys
PlorseelapinghienHen yof GLANCHI me. strsithisgothers jveere!-e!
QUCotouciullle's fld


In [19]:
x = torch.randn((5,5))

In [20]:
B,T ,C = 4,8,32
x = torch.randn((B,T,C))

head_size = 16

key = nn.Linear(C , head_size , bias=False)
query = nn.Linear(C , head_size , bias=False)
value = nn.Linear(C , head_size , bias=False)

k = key(x)
q = query(x)
wei = q @ k.transpose(-2,-1)
v = value(x)

tril = torch.tril(torch.ones(T,T))

wei = wei.masked_fill(tril== 0 ,float('-inf'))
wei = F.softmax(wei,dim=-1)
out = wei @ x




In [50]:
torch.manual_seed(4)

B,T,C = 4,8,32
x = torch.randn(B,T,C)

head_size = 16
key = nn.Linear(C,head_size , bias=False)
query = nn.Linear(C,head_size , bias=False)
#print(key.weight.shape , query.weight)
k = key(x)
q = query(x)

wei = q @ k.transpose(-2,-1)

tril = torch.tril(torch.ones(T,T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril==0, float('-inf'))
wei = F.softmax(wei,dim=-1)
out = wei @ x
out.shape

torch.Size([4, 8, 32])

In [1]:
import torch
import torch.nn as nn

In [34]:
x = torch.randn(4,8,32)

layer1 = nn.Linear(32,8,bias=False)
print(layer1.weight.shape)
layer1(x).shape

torch.Size([8, 32])


torch.Size([4, 8, 8])